In [42]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pylab as plt
#%pip install seaborn
import seaborn as sns

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator


from sklearn import set_config

#%pip install dagshub
import mlflow
import mlflow.sklearn
import dagshub


from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor

import optuna


set_config(transform_output="pandas")
plt.style.use('fivethirtyeight')

C:\Users\khadk\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dagshub.init(repo_owner='Arej02', repo_name='swiggy_delivery_time_prediction', mlflow=True)

Accessing as Arej02

Initialized MLflow to track repo "Arej02/swiggy_delivery_time_prediction"

Repository Arej02/swiggy_delivery_time_prediction initialized!

In [3]:
swiggy=pd.read_csv("swiggy_cleaned.csv")

In [4]:
swiggy.sample(3)

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,time_taken,city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance
21139,GOARES02DEL01,37.0,NaN,15.585658,73.743606,15.595658,73.753606,2022-02-13,windy,low,...,20,GOA,13,2,sunday,1,5.0,10.0,morning,1.543875
25105,COIMBRES18DEL01,28.0,5.0,11.020910,76.940432,11.060910,76.980432,2022-03-30,fog,medium,...,27,COIMB,30,3,wednesday,0,5.0,15.0,afternoon,6.232194
6964,MYSRES13DEL03,25.0,4.8,12.310972,76.659264,12.400972,76.749264,2022-03-02,stormy,jam,...,28,MYS,2,3,wednesday,0,10.0,19.0,evening,13.989852


In [5]:
swiggy['distance'].describe()

count    41872.000000
mean         9.719296
std          5.602890
min          1.465067
25%          4.657655
50%          9.193014
75%         13.680920
max         20.969489
Name: distance, dtype: float64

In [6]:
def bin_distance(df:pd.DataFrame,num_col):
    distance_labels=['short','medium','long','very long']
    distance_bins=[1,6,12,18,24]
    df['distance_bins']=pd.cut(df[num_col],bins=distance_bins,labels=distance_labels,right=False)
    return df

In [7]:
bin_distance(swiggy,'distance')

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,city_name,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance,distance_bins
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,INDO,19,3,saturday,1,15.0,11.0,morning,3.025149,short
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,BANG,25,3,friday,0,5.0,19.0,evening,20.183530,very long
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,BANG,19,3,saturday,1,15.0,8.0,morning,1.552758,short
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,COIMB,5,4,tuesday,0,10.0,18.0,evening,7.790401,medium
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,CHEN,26,3,saturday,1,15.0,13.0,afternoon,6.210138,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45497,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,windy,high,...,JAP,24,3,thursday,0,10.0,11.0,morning,1.489846,short
45498,AGRRES16DEL01,21.0,4.6,NaN,NaN,NaN,NaN,2022-02-16,windy,jam,...,AGR,16,2,wednesday,0,15.0,19.0,evening,NaN,NaN
45499,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,cloudy,low,...,CHEN,11,3,friday,0,15.0,23.0,night,4.657195,short
45500,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,cloudy,high,...,COIMB,7,3,monday,0,5.0,13.0,afternoon,6.232393,medium


In [8]:
swiggy['order_time_hour'].describe()

count    43862.000000
mean        17.423966
std          4.817856
min          0.000000
25%         15.000000
50%         19.000000
75%         21.000000
max         23.000000
Name: order_time_hour, dtype: float64

In [9]:
def bin_time(df:pd.DataFrame,num_col):
    time_labels=['after_midnight','morning','afternoon','evening','night']
    time_bins=[0,6,12,17,20,24]
    df['order_time_bins']=pd.cut(df[num_col],bins=time_bins,labels=time_labels,right=False)
    return df

In [10]:
bin_time(swiggy,'order_time_hour')

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance,distance_bins,order_time_bins
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,19,3,saturday,1,15.0,11.0,morning,3.025149,short,morning
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,25,3,friday,0,5.0,19.0,evening,20.183530,very long,evening
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,19,3,saturday,1,15.0,8.0,morning,1.552758,short,morning
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,5,4,tuesday,0,10.0,18.0,evening,7.790401,medium,evening
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,26,3,saturday,1,15.0,13.0,afternoon,6.210138,medium,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45497,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,windy,high,...,24,3,thursday,0,10.0,11.0,morning,1.489846,short,morning
45498,AGRRES16DEL01,21.0,4.6,NaN,NaN,NaN,NaN,2022-02-16,windy,jam,...,16,2,wednesday,0,15.0,19.0,evening,NaN,NaN,evening
45499,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,cloudy,low,...,11,3,friday,0,15.0,23.0,night,4.657195,short,night
45500,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,cloudy,high,...,7,3,monday,0,5.0,13.0,afternoon,6.232393,medium,afternoon


In [11]:
df1=swiggy.copy().dropna()
df1

,rider_id,age,ratings,restaurant_latitude,restaurant_longitude,delivery_latitude,delivery_longitude,order_date,weather,traffic,...,order_day,order_month,order_day_of_week,is_weekend,pickup_time_minutes,order_time_hour,order_time_of_day,distance,distance_bins,order_time_bins
0,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,sunny,high,...,19,3,saturday,1,15.0,11.0,morning,3.025149,short,morning
1,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,stormy,jam,...,25,3,friday,0,5.0,19.0,evening,20.183530,very long,evening
2,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,sandstorms,low,...,19,3,saturday,1,15.0,8.0,morning,1.552758,short,morning
3,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,sunny,medium,...,5,4,tuesday,0,10.0,18.0,evening,7.790401,medium,evening
4,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,cloudy,high,...,26,3,saturday,1,15.0,13.0,afternoon,6.210138,medium,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45496,RANCHIRES16DEL01,35.0,4.2,23.371292,85.327872,23.481292,85.437872,2022-03-08,windy,jam,...,8,3,tuesday,0,10.0,21.0,night,16.600272,long,night
45497,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,2022-03-24,windy,high,...,24,3,thursday,0,10.0,11.0,morning,1.489846,short,morning
45499,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,2022-03-11,cloudy,low,...,11,3,friday,0,15.0,23.0,night,4.657195,short,night
45500,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,2022-03-07,cloudy,high,...,7,3,monday,0,5.0,13.0,afternoon,6.232393,medium,afternoon


In [12]:
columns_to_drop=[
    'rider_id',
    'restaurant_latitude',
    'restaurant_longitude',	
    'delivery_latitude',
    'delivery_longitude',
    'order_date',
    'order_time_of_day',
    "order_time_hour",
    "order_day",
    "city_name",
    "order_day_of_week",
    "order_month"
]

In [13]:
df1.drop(columns=columns_to_drop,inplace=True)

In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38064 entries, 0 to 45501
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   age                  38064 non-null  float64 
 1   ratings              38064 non-null  float64 
 2   weather              38064 non-null  object  
 3   traffic              38064 non-null  object  
 4   vehicle_condition    38064 non-null  int64   
 5   type_of_order        38064 non-null  object  
 6   type_of_vehicle      38064 non-null  object  
 7   multiple_deliveries  38064 non-null  float64 
 8   festival             38064 non-null  object  
 9   city_type            38064 non-null  object  
 10  time_taken           38064 non-null  int64   
 11  is_weekend           38064 non-null  int64   
 12  pickup_time_minutes  38064 non-null  float64 
 13  distance             38064 non-null  float64 
 14  distance_bins        38064 non-null  category
 15  order_time_bins      380

In [15]:
df1.isnull().mean()

age                    0.0
ratings                0.0
weather                0.0
traffic                0.0
vehicle_condition      0.0
type_of_order          0.0
type_of_vehicle        0.0
multiple_deliveries    0.0
festival               0.0
city_type              0.0
time_taken             0.0
is_weekend             0.0
pickup_time_minutes    0.0
distance               0.0
distance_bins          0.0
order_time_bins        0.0
dtype: float64

In [16]:
df1.duplicated().sum()

np.int64(0)

In [17]:
X=df1.drop(columns=['time_taken'])
X

,age,ratings,weather,traffic,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city_type,is_weekend,pickup_time_minutes,distance,distance_bins,order_time_bins
0,37.0,4.9,sunny,high,2,snack,motorcycle,0.0,no,urban,1,15.0,3.025149,short,morning
1,34.0,4.5,stormy,jam,2,snack,scooter,1.0,no,metropolitian,0,5.0,20.183530,very long,evening
2,23.0,4.4,sandstorms,low,0,drinks,motorcycle,1.0,no,urban,1,15.0,1.552758,short,morning
3,38.0,4.7,sunny,medium,0,buffet,motorcycle,1.0,no,metropolitian,0,10.0,7.790401,medium,evening
4,32.0,4.6,cloudy,high,1,snack,scooter,1.0,no,metropolitian,1,15.0,6.210138,medium,afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45496,35.0,4.2,windy,jam,2,drinks,motorcycle,1.0,no,metropolitian,0,10.0,16.600272,long,night
45497,30.0,4.8,windy,high,1,meal,motorcycle,0.0,no,metropolitian,0,10.0,1.489846,short,morning
45499,30.0,4.9,cloudy,low,1,drinks,scooter,0.0,no,metropolitian,0,15.0,4.657195,short,night
45500,20.0,4.7,cloudy,high,0,snack,motorcycle,1.0,no,metropolitian,0,5.0,6.232393,medium,afternoon


In [18]:
Y=df1['time_taken']
Y

0        24
1        33
2        26
3        21
4        30
         ..
45496    33
45497    32
45499    16
45500    26
45501    36
Name: time_taken, Length: 38064, dtype: int64

In [19]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=10)

In [20]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (30451, 15)
The shape of test data is (7613, 15)


In [21]:
numerical_col=[cols for cols in df1.columns if df1[cols].dtype in ['int64','float64'] and cols !='time_taken']
numerical_col

['age',
 'ratings',
 'vehicle_condition',
 'multiple_deliveries',
 'is_weekend',
 'pickup_time_minutes',
 'distance']

In [22]:
categorical_col=[cols for cols in df1.columns if df1[cols].dtype in ['object','category']]
categorical_col

['weather',
 'traffic',
 'type_of_order',
 'type_of_vehicle',
 'festival',
 'city_type',
 'distance_bins',
 'order_time_bins']

In [23]:
nominal_col=['weather','type_of_order','type_of_vehicle','festival','distance_bins','order_time_bins']
ordinal_col=['traffic','city_type']

In [24]:
len(numerical_col+nominal_col+ordinal_col)

15

In [25]:
df1['traffic'].unique()

array(['high', 'jam', 'low', 'medium'], dtype=object)

In [26]:
df1['city_type'].unique()

array(['urban', 'metropolitian', 'semi-urban'], dtype=object)

In [27]:
traffic_groups=['low','medium','high','jam']
city_groups=['semi-urban','urban','metropolitian']

In [28]:
preprocessor=ColumnTransformer(
    [
        ("scaler",StandardScaler(),numerical_col),
        ("ohe",OneHotEncoder(drop="first",sparse_output=False,handle_unknown="ignore"),nominal_col),
        ("ordinal",OrdinalEncoder(categories=[traffic_groups,city_groups]),ordinal_col)
    ],remainder='passthrough'
)
preprocessor.set_output(transform="pandas")

ColumnTransformer(remainder='passthrough',
                  transformers=[('scaler', StandardScaler(),
                                 ['age', 'ratings', 'vehicle_condition',
                                  'multiple_deliveries', 'is_weekend',
                                  'pickup_time_minutes', 'distance']),
                                ('ohe',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['weather', 'type_of_order', 'type_of_vehicle',
                                  'festival', 'distance_bins',
                                  'order_time_bins']),
                                ('ordinal',
                                 OrdinalEncoder(categories=[['low', 'medium',
                                                             'high', 'jam'],
                                                            ['semi-urban',
                                                             'urban',
                                                             'metropolitian']]),
                                 ['traffic', 'city_type'])])

In [29]:
base_model=RandomForestRegressor()

In [30]:
pipeline1=Pipeline([
    ("preprocessor",preprocessor),
    ("base_model",base_model)
])

In [31]:
reg=TransformedTargetRegressor(
    regressor=pipeline1,
    transformer=PowerTransformer()
)

In [32]:
reg.fit(X_train,Y_train)

C:\Users\khadk\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but PowerTransformer was fitted without feature names
  warnings.warn(


TransformedTargetRegressor(regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(remainder='passthrough',
                                                                        transformers=[('scaler',
                                                                                       StandardScaler(),
                                                                                       ['age',
                                                                                        'ratings',
                                                                                        'vehicle_condition',
                                                                                        'multiple_deliveries',
                                                                                        'is_weekend',
                                                                                        'pickup_time_minutes',
                                                                                        'distance']),
                                                                                      ('ohe',
                                                                                       OneHotEncoder(drop='first',
                                                                                                     handle_unknown='ignore',
                                                                                                     sparse_output=False),
                                                                                       ['weather',
                                                                                        'type_of_order',
                                                                                        'type_of_vehicle',
                                                                                        'festival',
                                                                                        'distance_bins',
                                                                                        'order_time_bins']),
                                                                                      ('ordinal',
                                                                                       OrdinalEncoder(categories=[['low',
                                                                                                                   'medium',
                                                                                                                   'high',
                                                                                                                   'jam'],
                                                                                                                  ['semi-urban',
                                                                                                                   'urban',
                                                                                                                   'metropolitian']]),
                                                                                       ['traffic',
                                                                                        'city_type'])])),
                                                     ('base_model',
                                                      RandomForestRegressor())]),
                           transformer=PowerTransformer())

In [33]:
y_pred_test=reg.predict(X_test)
y_pred_train=reg.predict(X_train)

In [34]:
print(f"The train mean absolute error is:{round(mean_absolute_error(Y_train,y_pred_train),2)} min")
print(f"The test mean absolute error is:{round(mean_absolute_error(Y_test,y_pred_test),2)} min")

The train mean absolute error is:1.15 min
The test mean absolute error is:3.14 min


In [35]:
print(f"The train r2 score is:{round(r2_score(Y_train,y_pred_train),2)}")
print(f"The test r2 score is:{round(r2_score(Y_test,y_pred_test),2)}")

The train r2 score is:0.98
The test r2 score is:0.82


In [37]:
pipeline2=Pipeline([
    ("preprocessor",preprocessor),
])

In [38]:
X_train_trans=pipeline2.fit_transform(X_train)
X_test_trans=pipeline2.transform(X_test)

In [39]:
X_train_trans

,scaler__age,scaler__ratings,scaler__vehicle_condition,scaler__multiple_deliveries,scaler__is_weekend,scaler__pickup_time_minutes,scaler__distance,ohe__weather_fog,ohe__weather_sandstorms,ohe__weather_stormy,...,ohe__festival_yes,ohe__distance_bins_medium,ohe__distance_bins_short,ohe__distance_bins_very long,ohe__order_time_bins_afternoon,ohe__order_time_bins_evening,ohe__order_time_bins_morning,ohe__order_time_bins_night,ordinal__traffic,ordinal__city_type
42620,-1.319586,0.528127,0.002935,0.437621,-0.615678,0.006393,0.198212,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
6159,0.936057,-0.104873,-1.221503,0.437621,-0.615678,1.230791,-1.468722,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
18429,0.415524,0.844627,1.227374,-1.310051,1.624227,0.006393,-0.933730,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
2206,-1.319586,0.211627,1.227374,0.437621,-0.615678,0.006393,-1.199370,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0
11565,0.762546,-0.737873,0.002935,-1.310051,-0.615678,1.230791,-0.434219,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12203,-0.278520,0.211627,1.227374,0.437621,1.624227,0.006393,-0.633418,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0
11198,-0.452031,1.161127,0.002935,0.437621,-0.615678,1.230791,-0.078964,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
33459,1.109568,-2.636872,1.227374,2.185294,1.624227,-1.218005,1.861649,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0
34862,-0.625542,0.844627,0.002935,-1.310051,1.624227,-1.218005,-1.210381,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


#### Model Selection: Optuna

In [48]:
def objective(trial):
    model_name=trial.suggest_categorical("model",["SVM","LGBM","RF","GB"])

    if model_name=="SVM":
        kernel_svm=trial.suggest_categorical("kernel_svm",["linear","poly","rbf"])
        if kernel_svm=="linear":
            c_linear=trial.suggest_float("c_linear",0,10)
            base_model=SVR(C=c_linear,kernel="linear")
        
        elif kernel_svm == "poly":
                c_poly = trial.suggest_float("c_poly",0,10)
                degree_poly = trial.suggest_int("degree_poly",1,5)
                base_model = SVR(C=c_poly,degree=degree_poly,
                            kernel="poly")

        else:
                c_rbf = trial.suggest_float("c_rbf",0,100)
                gamma_rbf = trial.suggest_float("gamma_rbf",0,10)
                base_model = SVR(C=c_rbf,gamma=gamma_rbf,
                            kernel="rbf")

    elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,200)
            max_depth_rf = trial.suggest_int("max_depth_rf",2,20)
            base_model = RandomForestRegressor(n_estimators=n_estimators_rf,
                                        max_depth=max_depth_rf,
                                        random_state=42,
                                        n_jobs=-1)

    elif model_name == "GB":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,200)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",0,1)
            max_depth_gb = trial.suggest_int("max_depth_gb",2,20)
            base_model = GradientBoostingRegressor(n_estimators=n_estimators_gb,
                                                learning_rate=learning_rate_gb,
                                                max_depth=max_depth_gb,
                                                random_state=42)

    elif model_name == "LGBM":
            n_estimators_lgbm = trial.suggest_int("n_estimators_lgbm",10,200)
            learning_rate_lgbm = trial.suggest_float("learning_rate_lgbm",0.1,0.5)
            max_depth_lgbm = trial.suggest_int("max_depth_lgbm",2,20)
            base_model = LGBMRegressor(n_estimators=n_estimators_lgbm,
                                    learning_rate=learning_rate_lgbm,
                                    max_depth=max_depth_lgbm,
                                    random_state=42)


    model=TransformedTargetRegressor(
        regressor=base_model,
        transformer=PowerTransformer("yeo-johnson")
    )

    model.fit(X_train_trans,Y_train)
    y_pred=model.predict(X_test_trans)
    error=mean_absolute_error(Y_test,y_pred)

    return error

In [49]:
study=optuna.create_study(direction="minimize",study_name="model_selection")
study.optimize(objective,n_trials=30,n_jobs=-1)

[I 2025-09-17 16:26:30,490] A new study created in memory with name: model_selection
[I 2025-09-17 16:26:32,430] Trial 6 finished with value: 3.139867302273508 and parameters: {'model': 'RF', 'n_estimators_rf': 17, 'max_depth_rf': 16}. Best is trial 6 with value: 3.139867302273508.
[I 2025-09-17 16:26:32,882] Trial 0 finished with value: 4.0631405323523 and parameters: {'model': 'RF', 'n_estimators_rf': 121, 'max_depth_rf': 6}. Best is trial 6 with value: 3.139867302273508.
[I 2025-09-17 16:26:34,966] Trial 5 finished with value: 3.077727972083757 and parameters: {'model': 'RF', 'n_estimators_rf': 183, 'max_depth_rf': 13}. Best is trial 5 with value: 3.077727972083757.
[I 2025-09-17 16:26:35,982] Trial 10 finished with value: 4.0614770749656905 and parameters: {'model': 'RF', 'n_estimators_rf': 112, 'max_depth_rf': 6}. Best is trial 5 with value: 3.077727972083757.
[I 2025-09-17 16:26:35,993] Trial 2 finished with value: 3.3259097322996016 and parameters: {'model': 'LGBM', 'n_estimator

In [50]:
study.best_value

3.0643365279504735

In [51]:
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'model': 'LGBM', 'n_estimators_lgbm': 85, 'learning_rate_lgbm': 0.10244999821440022, 'max_depth_lgbm': 20}
Best trial accuracy: 3.0643365279504735


In [57]:
lgm_params={
    'n_estimators': 85, 
    'learning_rate': 0.10244999821440022, 
    'max_depth': 20
}

In [58]:
lgm=LGBMRegressor(**lgm_params)

In [59]:
reg2=TransformedTargetRegressor(
    regressor=lgm,
    transformer=PowerTransformer()
)

In [60]:
reg2.fit(X_train_trans,Y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001006 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 362
[LightGBM] [Info] Number of data points in the train set: 30451, number of used features: 27
[LightGBM] [Info] Start training from score 0.000000


C:\Users\khadk\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but PowerTransformer was fitted without feature names
  warnings.warn(


TransformedTargetRegressor(regressor=LGBMRegressor(learning_rate=0.10244999821440022,
                                                   max_depth=20,
                                                   n_estimators=85),
                           transformer=PowerTransformer())

In [68]:
y_pred_test=reg2.predict(X_test_trans)
y_pred_train=reg2.predict(X_train_trans)

In [69]:
print(f"The train mean absolute error is:{round(mean_absolute_error(Y_train,y_pred_train),2)} min")
print(f"The test mean absolute error is:{round(mean_absolute_error(Y_test,y_pred_test),2)} min")

The train mean absolute error is:2.96 min
The test mean absolute error is:3.06 min


In [70]:
print(f"The train r2 score is:{round(r2_score(Y_train,y_pred_train),2)}")
print(f"The test r2 score is:{round(r2_score(Y_test,y_pred_test),2)}")

The train r2 score is:0.85
The test r2 score is:0.83


In [72]:
scores = cross_val_score(reg2,
                         X_train_trans,
                         Y_train,
                         scoring="neg_mean_absolute_error",
                         cv=5,n_jobs=-1).mean()
print(-scores)

3.0607996187198334


In [74]:
scores = cross_val_score(reg2,
                         X_train_trans,
                         Y_train,
                         scoring="r2",
                         cv=5,n_jobs=-1).mean()
print(scores)

0.8337842746395869
